In [1]:
import numpy as np
import glob
import scipy.io as sio
import torch
from torch import nn
import random
import csv
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import yaml
from evaluate import error
import time
import re
from syn_DI_dataset import make_dataset, make_dataloader

### Loda Data

In [2]:
dataset_root = 'd:\Data\My_MMFi_Data\MMFi_Dataset'
with open('config.yaml', 'r') as fd:
    config = yaml.load(fd, Loader=yaml.FullLoader)

train_dataset, val_dataset = make_dataset(dataset_root, config)

['depth', 'wifi-csi']
S02 ['A01', 'A02', 'A03', 'A04', 'A06', 'A08', 'A09', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A26']
S03 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A17', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S05 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A19', 'A20', 'A21', 'A24', 'A25']
S06 ['A01', 'A03', 'A04', 'A06', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S08 ['A01', 'A02', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A13', 'A14', 'A17', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S09 ['A01', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A17', 'A18', 'A20', 'A21', 'A24', 'A25', 'A27']
S11 ['A01', 'A02', 'A03', 'A05', 'A06', 'A08', 'A10', 'A11', 'A12', 'A13', 'A16', 'A17', 'A18'

In [3]:
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.

    dict_keys(['modality', 'scene', 'subject', 'action', 'idx', 'output', 
    'input_rgb', 'input_depth', 'input_lidar', 'input_mmwave'])
    '''
    ## get sequence lengths
    # for t in batch:
    #     print(t.keys())
    #     print(a)
    # #     # print(t[0].shape,t[1].shape)
    kpts = []
    [kpts.append(np.array(t['output'])) for t in batch]
    kpts = torch.FloatTensor(np.array(kpts))

    # lengths = torch.tensor([t['input_mmwave'].shape[0] for t in batch ])

    # # rgb
    # rgb_data = np.array([(t['input_rgb']) for t in batch ])
    # rgb_data = torch.FloatTensor(rgb_data).permute(0,3,1,2)

    # depth
    depth_data = np.array([(t['input_depth']) for t in batch ])
    depth_data = torch.FloatTensor(depth_data).permute(0,3,1,2)

    # # mmwave
    # ## padd
    # mmwave_data = [torch.Tensor(t['input_mmwave']) for t in batch ]
    # mmwave_data = torch.nn.utils.rnn.pad_sequence(mmwave_data)
    # ## compute mask
    # mmwave_data = mmwave_data.permute(1,0,2)

    # # lidar
    # ## padd
    # lidar_data = [torch.Tensor(t['input_lidar']) for t in batch ]
    # lidar_data = torch.nn.utils.rnn.pad_sequence(lidar_data)
    # ## compute mask
    # lidar_data = lidar_data.permute(1,0,2)

    # wifi-csi
    wifi_data = np.array([(t['input_wifi-csi']) for t in batch ])
    wifi_data = torch.FloatTensor(wifi_data)
    "要改"
    exist_list = [False, True, False, False, True]

    # return rgb_data, depth_data, lidar_data, mmwave_data, wifi_data, kpts, modality_list
    return depth_data, wifi_data, kpts, exist_list

In [4]:
rng_generator = torch.manual_seed(config['init_rand_seed'])
train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)
val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)

#### check random seed

In [17]:
print(len(train_loader.dataset)/16)

12991.5


In [5]:
avg_time = 0
for epoch in range(3):
    # random.seed(config['modality_existances']['train_random_seed'])
    i= 0
    for data in val_loader:
        # start_time = time.time()
        input_1, input2_, kpts, exist_list = data
        # epoch_time = time.time() - start_time
        # print(rgb_data[0].shape, depth_data[0].shape, lidar_data[0].shape, mmwave_data[0].shape, wifi_data[0].shape,kpts.shape, lengths.shape)
        print(input_1.shape, input2_.shape, kpts.shape)
        # print('epoch_time: ', epoch_time)
        # avg_time += epoch_time
        # print(rgb_data, depth_data, lidar_data, mmwave_data, wifi_data, kpts, modality_list)
        print(exist_list)
        i += 1
        if i > 1:
            print('............................................................................................')
            break
    # print('tot_time: ', avg_time)
    # print('avg_time: ', avg_time / (i+1))

torch.Size([16, 3, 480, 640]) torch.Size([16, 3, 114, 10]) torch.Size([16, 17, 3])
[False, True, False, False, True]
torch.Size([16, 3, 480, 640]) torch.Size([16, 3, 114, 10]) torch.Size([16, 17, 3])
[False, True, False, False, True]
torch.Size([16, 3, 480, 640]) torch.Size([16, 3, 114, 10]) torch.Size([16, 17, 3])
[False, True, False, False, True]
torch.Size([16, 3, 480, 640]) torch.Size([16, 3, 114, 10]) torch.Size([16, 17, 3])
[False, True, False, False, True]
torch.Size([16, 3, 480, 640]) torch.Size([16, 3, 114, 10]) torch.Size([16, 17, 3])
[False, True, False, False, True]
torch.Size([16, 3, 480, 640]) torch.Size([16, 3, 114, 10]) torch.Size([16, 17, 3])
[False, True, False, False, True]
............................................................................................
torch.Size([16, 3, 480, 640]) torch.Size([16, 3, 114, 10]) torch.Size([16, 17, 3])
[False, True, False, False, True]
torch.Size([16, 3, 480, 640]) torch.Size([16, 3, 114, 10]) torch.Size([16, 17, 3])
[Fals

### Model

In [10]:
'model without pos encoding'
from baseline_model import dual_model

# model = model(['RGB'])
model = dual_model(['Depth','Wifi'])
model.cuda()

# rgb_data = torch.randn(16, 3, 480, 640).cuda()
# # depth_data = torch.randn(16, 3, 480, 640).cuda()
# # mmwave_data = torch.randn(16, 67, 5).cuda()
# # lidar_data = torch.randn(16, 1432, 3).cuda()
# wifi_data = torch.randn(16, 3, 114, 10).cuda()
# modality_list = [True, False, False, False, True]
# # modality_list = [True, True, True, True, True]


# # out = model(rgb_data, depth_data,  mmwave_data, lidar_data, wifi_data, modality_list)
# out = model(rgb_data, wifi_data, modality_list)
# print(out.shape)

dual_model(
  (feature_extractor): Dual_feature_extrator(
    (depth_extractor): depth_feature_extractor(
      (part): Sequential(
        (0): Sequential(
          (0): Conv2d(3, 3, kernel_size=(14, 14), stride=(2, 2))
          (1): ReLU()
          (2): Conv2d(3, 3, kernel_size=(5, 56), stride=(1, 1))
          (3): ReLU()
          (4): Conv2d(3, 3, kernel_size=(5, 23), stride=(1, 1))
          (5): ReLU()
          (6): Conv2d(3, 16, kernel_size=(3, 14), stride=(1, 1))
        )
        (1): Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (5): Sequential(
          (0): Block(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, aff

In [5]:
'model with pos encoding'
from MI_model_copy import modality_invariant_model

model = modality_invariant_model()
model.cuda()

# rgb_data = torch.randn(16, 3, 480, 640).cuda()
# depth_data = torch.randn(16, 3, 480, 640).cuda()
# lidar_data = torch.randn(16, 1432, 3).cuda()
# mmwave_data = torch.randn(16, 67, 5).cuda()
# wifi_data = torch.randn(16, 3, 114, 10).cuda()
# # modality_list = [False, False, False, False, False]
# modality_list = [True, True, True, True, True]


# out = model(rgb_data, depth_data,  mmwave_data, lidar_data, wifi_data, modality_list)

# print(out.shape)

modality_invariant_model(
  (feature_extractor): feature_extrator(
    (rgb_extractor): rgb_feature_extractor(
      (part): Sequential(
        (0): Sequential(
          (0): Conv2d(3, 3, kernel_size=(14, 14), stride=(2, 2))
          (1): ReLU()
          (2): Conv2d(3, 3, kernel_size=(5, 56), stride=(1, 1))
          (3): ReLU()
          (4): Conv2d(3, 3, kernel_size=(5, 23), stride=(1, 1))
          (5): ReLU()
          (6): Conv2d(3, 16, kernel_size=(3, 14), stride=(1, 1))
        )
        (1): Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (5): Sequential(
          (0): Block(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1

### training 

In [11]:
def test(model, tensor_loader, criterion1, criterion2, device):
    model.eval()
    test_mpjpe = 0
    test_pampjpe = 0
    test_mse = 0
    # random.seed(config['modality_existances']['val_random_seed'])
    for data in tqdm(tensor_loader):
        # start_time = time.time()
        input_1, input_2, kpts, exist_list = data
        input_1 = input_1.to(device)
        input_2 = input_2.to(device)
        kpts.to(device)
        labels = kpts.type(torch.FloatTensor)
        outputs = model(input_1, input_2, exist_list)
        outputs = outputs.type(torch.FloatTensor)
        outputs.to(device)
        # t2 = time.time()
        # forward_time = t2 - t1
        test_mse += criterion1(outputs,labels).item() * input_1.size(0)

        outputs = outputs.detach().numpy()
        labels = labels.detach().numpy()
        
        mpjpe, pampjpe = criterion2(outputs,labels)
        test_mpjpe += mpjpe.item() * input_1.size(0)
        test_pampjpe += pampjpe.item() * input_1.size(0)
        # t3 = time.time()
        # record_time = t3 - t2
        # print('load_time: ', load_time)
        # print('forward_time: ', forward_time)
        # print('record_time: ', record_time)
    test_mpjpe = test_mpjpe/len(tensor_loader.dataset)
    test_pampjpe = test_pampjpe/len(tensor_loader.dataset)
    test_mse = test_mse/len(tensor_loader.dataset)
    print("mse: {:.8f}, mpjpe: {:.8f}, pampjpe: {:.8f}".format(float(test_mse), float(test_mpjpe),float(test_pampjpe)))
    return test_mpjpe

In [11]:
train_criterion = nn.MSELoss()
test_criterion = error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
test(model, val_loader, train_criterion, test_criterion, device)

  1%|          | 39/3403 [00:12<18:28,  3.03it/s] 


KeyboardInterrupt: 

In [12]:
def train(model, train_loader, test_loader, num_epochs, learning_rate, train_criterion, test_criterion, device):
    optimizer = torch.optim.AdamW(
        [
                {'params': model.regression_head.parameters()}
            ],
        lr = learning_rate
    )
    # optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[20,40],gamma=0.1)
    "要改"
    parameter_dir = './baseline_weights/Dual/Depth_Wifi.pt'
    best_test_mpjpe = 100
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        epoch_accuracy = 0
        # random.seed(epoch)
        num_iter = 400
        for i, data in enumerate(tqdm(train_loader)):
            if i < num_iter:
                input_1, input_2, kpts, exist_list = data
                input_1 = input_1.to(device)
                input_2 = input_2.to(device)
                labels = kpts.to(device)
                labels = labels.type(torch.FloatTensor)
                
                optimizer.zero_grad()
                outputs = model(input_1, input_2, exist_list)
                # print(outputs)
                outputs = outputs.to(device)
                outputs = outputs.type(torch.FloatTensor)
                loss = train_criterion(outputs,labels)
                if loss == float('nan'):
                    print('nan')
                    print(outputs)
                    print(labels)
                    
                loss.backward()
                # print(length)
                # print("loss is ", loss.item())
                optimizer.step()
                
                epoch_loss += loss.item() * input_1.size(0)
            else:
                break
            # print("epoch loss is ", epoch_loss)
        # epoch_loss = epoch_loss/len(train_loader.dataset)
        "要改"
        epoch_loss = epoch_loss/(input_1.size(0)*num_iter)
        print('Epoch: {}, Loss: {:.8f}'.format(epoch, epoch_loss))
        if (epoch+1) % 5 == 0:
            test_mpjpe = test(
                model=model,
                tensor_loader=test_loader,
                criterion1 = train_criterion,
                criterion2 = test_criterion,
                device= device
            )
            # if test_mpjpe <= best_test_mpjpe:
            #     print(f"best test mpjpe is:{test_mpjpe}")
            #     best_test_mpjpe = test_mpjpe
            #     torch.save(model.state_dict(), parameter_dir)
        # scheduler.step()
    torch.save(model.state_dict(), parameter_dir)
    return

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device) 

cuda:0


In [14]:
train_criterion = nn.MSELoss()
test_criterion = error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.load_state_dict(torch.load('./pre-train_weights/lidar_p1_random.pt'))
model.to(device)
train(
    model=model,
    train_loader= train_loader,
     test_loader= val_loader,    
    num_epochs= 35,
    learning_rate=1e-3,
    train_criterion = train_criterion,
    test_criterion = test_criterion,
    device=device 
        )

  0%|          | 0/12991 [00:00<?, ?it/s]c:\Users\Chen_Xinyan\anaconda3\envs\pytorch\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
  3%|▎         | 400/12991 [18:28<9:41:41,  2.77s/it] 


Epoch: 0, Loss: 0.14043264


  3%|▎         | 400/12991 [17:11<9:01:17,  2.58s/it] 


Epoch: 1, Loss: 0.03195373


  3%|▎         | 400/12991 [16:04<8:26:00,  2.41s/it] 


...

  3%|▎         | 400/12991 [08:49<4:37:56,  1.32s/it] 


Epoch: 34, Loss: 0.00868466


100%|██████████| 3403/3403 [24:49<00:00,  2.29it/s]  

mse: 0.01054933, mpjpe: 0.15550875, pampjpe: 0.08150853


: 